In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import datetime as dt
import pandas_datareader.data as web
import timeit

pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 50)
pd.options.display.float_format = '{:,.4f}'.format

In [2]:
helper_dir = '../helper'

%run {helper_dir}/utils.py
%run {helper_dir}/rolling_funcs.py

## Make some data

In [3]:
num_sec_dfs, num_periods = 2, 480

dates = pd.date_range('1995-12-31', periods=num_periods, freq='M', name='Date')
stocks = pd.DataFrame(data=np.random.rand(num_periods, num_sec_dfs), index=dates,
                      columns=['s{:04d}'.format(i) for i in range(num_sec_dfs)]).pct_change().dropna()
market = pd.DataFrame(data=np.random.rand(num_periods), index=dates, 
                      columns= ['Market']).pct_change().dropna()
rets = stocks.join(market)

In [4]:
rets.head()

,s0000,s0001,Market
Date,,,
1996-01-31,-0.6248,0.1202,-0.9094
1996-02-29,1.2975,-0.1266,17.9019
1996-03-31,0.1899,0.4531,-0.0601
1996-04-30,0.7448,-0.0165,-0.6930
1996-05-31,-0.9324,-0.3584,-0.0100


In [5]:
rets.tail()

,s0000,s0001,Market
Date,,,
2035-07-31,-0.1576,-0.6148,4.0539
2035-08-31,0.4244,1.7399,0.5686
2035-09-30,-0.5073,0.1153,-0.4440
2035-10-31,0.0616,-0.7521,0.8185
2035-11-30,-0.2030,0.2022,-0.1706


## Input

In [6]:
ndays = 120

## Check our `roll()` and `groll()` give the same results as `df.rolling().mean()`

In [7]:
means_true = rets.rolling(ndays).mean()
means_01 = roll(rets, ndays).mean()
means_02 = pd.concat([pd.DataFrame([subdf.mean()], index=[subdf.index[-1]]) for subdf in groll(rets, ndays)])

In [8]:
print(means_true.dropna().head(), '\n\n')
print(means_01.head(), '\n\n')
print(means_02.head(), '\n\n')

            s0000  s0001   Market
Date                             
2005-12-31 3.3101 2.8087 119.6283
2006-01-31 3.3098 2.8114 119.6446
2006-02-28 3.3199 2.8121 119.4887
2006-03-31 3.3162 2.8051 119.4829
2006-04-30 3.3143 2.7972 119.6159 


            s0000  s0001   Market
Date                             
2005-12-31 3.3101 2.8087 119.6283
2006-01-31 3.3098 2.8114 119.6446
2006-02-28 3.3199 2.8121 119.4887
2006-03-31 3.3162 2.8051 119.4829
2006-04-30 3.3143 2.7972 119.6159 


            s0000  s0001   Market
2005-12-31 3.3101 2.8087 119.6283
2006-01-31 3.3098 2.8114 119.6446
2006-02-28 3.3199 2.8121 119.4887
2006-03-31 3.3162 2.8051 119.4829
2006-04-30 3.3143 2.7972 119.6159 




In [9]:
print(means_true.tail(), '\n\n')
print(means_01.tail(), '\n\n')
print(means_02.tail(), '\n\n')

            s0000  s0001  Market
Date                            
2035-07-31 4.1927 2.7028  3.6594
2035-08-31 4.1883 2.7139  3.6628
2035-09-30 4.1854 2.7226  3.6649
2035-10-31 4.1882 2.6956  3.6698
2035-11-30 4.1838 2.6873  3.6620 


            s0000  s0001  Market
Date                            
2035-07-31 4.1927 2.7028  3.6594
2035-08-31 4.1883 2.7139  3.6628
2035-09-30 4.1854 2.7226  3.6649
2035-10-31 4.1882 2.6956  3.6698
2035-11-30 4.1838 2.6873  3.6620 


            s0000  s0001  Market
2035-07-31 4.1927 2.7028  3.6594
2035-08-31 4.1883 2.7139  3.6628
2035-09-30 4.1854 2.7226  3.6649
2035-10-31 4.1882 2.6956  3.6698
2035-11-30 4.1838 2.6873  3.6620 




In [10]:
print(equal(means_true, means_01))
print(equal(means_true, means_02))

True
True


## Check our `roll()` and `groll()` give the same results as `df.rolling().std(ddof=1)`

In [11]:
sds_true = rets.rolling(ndays).std(ddof=1)
sds_01 = roll(rets, ndays).std(ddof=1)
sds_02 = pd.concat([pd.DataFrame([subdf.std(ddof=1)], index=[subdf.index[-1]]) for subdf in groll(rets, ndays)])

In [12]:
print(sds_true.dropna().head(), '\n\n')
print(sds_01.head(), '\n\n')
print(sds_02.head(), '\n\n')

             s0000   s0001     Market
Date                                 
2005-12-31 18.0846 12.4223 1,289.6654
2006-01-31 18.0846 12.4217 1,289.6639
2006-02-28 18.0838 12.4216 1,289.6774
2006-03-31 18.0845 12.4231 1,289.6780
2006-04-30 18.0848 12.4253 1,289.6663 


             s0000   s0001     Market
Date                                 
2005-12-31 18.0846 12.4223 1,289.6654
2006-01-31 18.0846 12.4217 1,289.6639
2006-02-28 18.0838 12.4216 1,289.6774
2006-03-31 18.0845 12.4231 1,289.6780
2006-04-30 18.0848 12.4253 1,289.6663 


             s0000   s0001     Market
2005-12-31 18.0846 12.4223 1,289.6654
2006-01-31 18.0846 12.4217 1,289.6639
2006-02-28 18.0838 12.4216 1,289.6774
2006-03-31 18.0845 12.4231 1,289.6780
2006-04-30 18.0848 12.4253 1,289.6663 




In [13]:
print(sds_true.tail(), '\n\n')
print(sds_01.tail(), '\n\n')
print(sds_02.tail(), '\n\n')

             s0000   s0001  Market
Date                              
2035-07-31 24.7435 13.4642 20.5394
2035-08-31 24.7442 13.4628 20.5388
2035-09-30 24.7447 13.4608 20.5384
2035-10-31 24.7442 13.4645 20.5377
2035-11-30 24.7450 13.4658 20.5389 


             s0000   s0001  Market
Date                              
2035-07-31 24.7435 13.4642 20.5394
2035-08-31 24.7442 13.4628 20.5388
2035-09-30 24.7447 13.4608 20.5384
2035-10-31 24.7442 13.4645 20.5377
2035-11-30 24.7450 13.4658 20.5389 


             s0000   s0001  Market
2035-07-31 24.7435 13.4642 20.5394
2035-08-31 24.7442 13.4628 20.5388
2035-09-30 24.7447 13.4608 20.5384
2035-10-31 24.7442 13.4645 20.5377
2035-11-30 24.7450 13.4658 20.5389 




In [14]:
print(equal(sds_true, sds_01))
print(equal(sds_true, sds_02))

True
True
